In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

2017-04-27 00:49:11,181 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Model 1 evaluation

In [2]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/skip_gram_simple_model_1.model')
model.init_sims(replace=True)

2017-04-27 00:49:11,769 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/skip_gram_simple_model_1.model
2017-04-27 00:49:12,994 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/skip_gram_simple_model_1.model.wv.* with mmap=None
2017-04-27 00:49:12,995 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/skip_gram_simple_model_1.model.wv.syn0.npy with mmap=None
2017-04-27 00:49:13,257 : INFO : setting ignored attribute syn0norm to None
2017-04-27 00:49:13,259 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/skip_gram_simple_model_1.model
2017-04-27 00:49:13,876 : INFO : precomputing L2-norms of word weight vectors


In [3]:
print(model.most_similar(positive=['rss', 'haha']))

[('hahahahahaha', 0.9640428423881531), ('hihi', 0.9617315530776978), ('heim', 0.9563144445419312), ('kkk"', 0.9561717510223389), ('pqp', 0.955443799495697), ('hein,', 0.9543150663375854), ('hahahah', 0.9540662169456482), ('*u*', 0.9540024399757385), ('dahora', 0.9532981514930725), ('hahahahah', 0.9530462026596069)]


In [4]:
num_features= 100

In [5]:
df = pd.read_csv('~/personality-normalized-word2vec.csv', encoding='utf-8')
df.shape

(1039, 186)

In [6]:
df.extraversion_m.value_counts()

0    529
1    510
Name: extraversion_m, dtype: int64

In [7]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

In [8]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['extraversion_m']
    evaluate_prediction(predictions, target)

In [9]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [10]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [11]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [12]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-04-27 00:50:01,088 : WARNING : cannot compute similarity with no input [u'FOREIGN']
2017-04-27 00:50:01,447 : WARNING : cannot compute similarity with no input []
2017-04-27 00:50:01,770 : WARNING : cannot compute similarity with no input []
2017-04-27 00:50:01,777 : WARNING : cannot compute similarity with no input []
2017-04-27 00:50:02,404 : WARNING : cannot compute similarity with no input []
2017-04-27 00:50:02,663 : WARNING : cannot compute similarity with no input []
2017-04-27 00:50:03,035 : WARNING : cannot compute similarity with no input [u'FOREIGN', u'FOREIGN', u'FOREIGN']
2017-04-27 00:50:03,051 : WARNING : cannot compute similarity with no input []
2017-04-27 00:50:03,853 : WARNING : cannot compute similarity with no input []
2017-04-27 00:50:03,941 : WARNING : cannot compute similarity with no input []
2017-04-27 00:50:04,170 : WARNING : cannot compute similarity with no input []
2017-04-27 00:50:04,331 : WARNING : cannot compute similarity with no input []
2017-04-

CPU times: user 3.99 s, sys: 9.35 ms, total: 4 s
Wall time: 3.98 s


In [13]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [14]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['extraversion_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   35.7s finished


CPU times: user 5.28 s, sys: 131 ms, total: 5.41 s
Wall time: 41.1 s


In [15]:
joblib.dump(model, '/home/bahbbc/workspace/masters-big5/extraversion-best-random-tree-model1.pkl')

['/home/bahbbc/workspace/masters-big5/extraversion-best-random-tree-model1.pkl']

In [16]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['extraversion_m']

In [17]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,100,42
1,80,90


In [18]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.56      0.70      0.62       142
          1       0.68      0.53      0.60       170

avg / total       0.62      0.61      0.61       312



In [19]:
accuracy_score(yt, yp)

0.60897435897435892